# Census Data

In [13]:
import cenpy
import pandas as pd
import geopandas as gpd
acs = cenpy.remote.APIConnection("ACSDT5Y2015")

In [17]:
censusRace = acs.query(
    cols=["NAME","B02001_001E","B02001_002E"],
    geo_unit="block group:*",
    geo_filter={
                "state" : "42", 
                "county" : "101"
               },
).rename(columns={"B02001_001E": "totalPop","B02001_002E": "whitePop"}, errors="raise")

In [28]:
censusRace2 = censusRace
censusRace2['whitePop'] = pd.to_numeric(censusRace2['whitePop'], errors='ignore')
censusRace2['totalPop'] = pd.to_numeric(censusRace2['totalPop'], errors='ignore')
censusRace2['whitePercep'] = censusRace2['whitePop'] / censusRace2['totalPop'] 
censusRace2['nonwhitePercep'] = 1- censusRace2['whitePercep']
censusRace2.head()

,NAME,totalPop,whitePop,state,county,tract,block group,whitePercep,nonwhitePercep
0,"Block Group 1, Census Tract 81.02, Philadelphi...",858,0,42,101,008102,1,0.000000,1.000000
1,"Block Group 5, Census Tract 81.02, Philadelphi...",894,11,42,101,008102,5,0.012304,0.987696
2,"Block Group 3, Census Tract 81.02, Philadelphi...",543,0,42,101,008102,3,0.000000,1.000000
3,"Block Group 2, Census Tract 248, Philadelphia ...",1109,26,42,101,024800,2,0.023445,0.976555
4,"Block Group 1, Census Tract 2, Philadelphia Co...",2331,911,42,101,000200,1,0.390819,0.609181


In [30]:
censusRace2.to_csv('censusRaceData.csv', index=False) 

# 311 Complaints

In [3]:
complaint311 = pd.read_csv ("downloaded_public_cases_fc.csv")

In [4]:
ndService=['Abandoned Vehicle','Graffiti Removal','Vacant House or Commercial','Street Light Outage','Traffic Signal Emergency','Dead Animal in Street', 'Street Defect','Vacant Lot Clean-Up', 'Alley Light Outage', 'Street Trees', 'Abandoned Bike',
       'Complaint (Streets)', 'Street Paving', 'Tree Dangerous']

In [17]:
complaint311Filter = complaint311.query('service_name in @ndService')
complaint311Filter = complaint311Filter[["service_request_id","service_name","agency_responsible","lat","lon"]].dropna()

In [18]:
complaint311Filter['geometry'] = gpd.points_from_xy(complaint311Filter['lon'], complaint311Filter['lat'])
complaint311Filter = gpd.GeoDataFrame(complaint311Filter, geometry='geometry', crs="EPSG:4326")

In [19]:
complaint311Filter.count()

service_request_id    64523
service_name          64523
agency_responsible    64523
lat                   64523
lon                   64523
geometry              64523
dtype: int64